In [3]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

2023-10-15 13:29:01.842269: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-15 13:29:01.842314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-15 13:29:01.843423: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-15 13:29:01.988625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 13:29:08.874824: W tensorflow/compiler/

In [4]:
df   = pd.read_csv('datasets/disaster.csv')
text = df['text'][2]

In [5]:
def prepare_json(text):
    feature_spec = {
        'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, 'utf-8')]))
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [6]:
def predict(text):
    response = requests.post(
        'http://localhost:8080/v1/models/disaster-tweets:predict',
        data = prepare_json(text)
    )

    predict = response.json()['predictions'][0][0]
    return predict

In [7]:
predict = predict(text)

print(f'{text}\n')
print(f'Tweet Asli ({predict})' if predict > 0.6 else f'Tweet Palsu ({predict})')

All residents asked to  ' shelter in place '  are being notified by officers .  No other evacuation or shelter in place orders are expected

Tweet Asli (0.856196046)
